In [1]:
sigma_at = 1.e5 #Pa
nu_e = 100.
omega_e = 0.6
e0 = 0.5

E_s = lambda sig, nu, om: nu * sigma_at * np.power(sig/sigma_at,om)

In [2]:
sigma_1 = np.linspace(sigma_at, 2.e5,1000) #Pa
sigma_2 = np.linspace(2.e5, 4.e5,1000) #Pa

NameError: name 'np' is not defined

In [3]:
def calc_strain(nu_e,omega_e):
    eps = np.array([0.])
    #Erstbelastung
    for i in range(1,len(sigma_1)):
        eps = np.append(eps,eps[-1]+(sigma_1[i]-sigma_1[i-1])/E_s(sigma_1[i-1],nu_e,omega_e))
    #Entlastung
    for i in range(len(sigma_1)-1,0,-1):
        eps = np.append(eps,eps[-1]-(sigma_1[i]-sigma_1[i-1])/E_s(sigma_1[i-1],nu_e*10,1))
    eps=np.append(eps,eps[-1])
    #Wiederbelastung
    for i in range(1,len(sigma_1)):
        eps = np.append(eps,eps[-1]+(sigma_1[i]-sigma_1[i-1])/E_s(sigma_1[i-1],nu_e*10,1))
    eps=np.append(eps,eps[-1])
    #Erstbelastung
    for i in range(1,len(sigma_2)):
        eps = np.append(eps,eps[-1]+(sigma_2[i]-sigma_2[i-1])/E_s(sigma_2[i-1],nu_e,omega_e))
    eps=np.append(eps,eps[-1]+(sigma_2[-1]-sigma_2[-2])/E_s(sigma_2[-2],nu_e,omega_e))
    return eps

In [4]:
def e(strain):
    return e0-strain*(1+e0)

In [1]:
def plot_interactive_curve():
    @interact(show_Ton = widgets.Checkbox(value=False, description=r'Ton'),
              show_Schluff = widgets.Checkbox(value=False, description=r'Schluff'),
              show_Sand_Kies = widgets.Checkbox(value=False, description=r'Sand und Kies')
    )
    def plot_curve(show_Ton,show_Schluff,show_Sand_Kies):
        
        eps_Ton = calc_strain(10,0.9)
        eps_Schluff = calc_strain(50,0.85)
        eps_Sand_Kies = calc_strain(400,0.65)        
        
        fig, ax = plt.subplots(ncols=2,figsize=(18,8))
        if (show_Ton):
            ax[0].plot(np.append(sigma_1,
                               np.append(np.append(np.flip(sigma_1),sigma_1),
                                         sigma_2))/1e3,
                  -eps_Ton*100)
        if (show_Schluff):
            ax[0].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              -eps_Schluff*100)
        if (show_Sand_Kies):
            ax[0].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              -eps_Sand_Kies*100)
        
        ticks =  ax[0].get_yticks()
        # set labels to absolute values and with integer representation
        ax[0].set_yticks(ticks)
        ax[0].set_yticklabels([round(abs(tick),3) for tick in ticks])
        ax[0].set_xlabel(r'$\sigma$ / kPa')
        ax[0].set_ylabel(r'$\epsilon$ / %')

        if (show_Ton):
            ax[1].plot(np.append(sigma_1,
                               np.append(np.append(np.flip(sigma_1),sigma_1),
                                         sigma_2))/1e3,
                  e(eps_Ton),label=r'Ton')
        if (show_Schluff):
            ax[1].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              e(eps_Schluff),label=r'Schluff')
        if (show_Sand_Kies):
            ax[1].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              e(eps_Sand_Kies),label=r'Sand und Kies')
        ax[1].set_xlabel(r'$\sigma$ / kPa')
        ax[1].set_ylabel(r'$e$')
        ax[1].set_xscale('log')
        if (show_Ton or show_Schluff or show_Sand_Kies):
            ax[1].legend()
        plt.show();

In [2]:
def plot_stiffness_input():
    sigma = np.linspace(10,1000,1000) #kPa
    E_s = lambda sig, Eref, sigref, om: Eref * np.power(sig/sigref,om) #kPa
    eps = lambda sig, Eref, sigref, om: sigref*np.log(sig/sigref)/Eref if om == 1 else sigref/((1-om)*Eref) * ( np.power(sig/sigref,1-om) -1 )
    loopifier = lambda w: w*(1-w) * 0.005
    gamma = 16.
    q0 = 300.
    ts = 25.
    a = 6.
    b = 4.
    fig, ax = plt.subplots(ncols=3,figsize=(18,8))
    ax[0].plot(sigma,eps(sigma,1000,10,0.9)*100)
    ax[0].set_xlabel(r"$\sigma'$ / kPa")
    ax[0].set_ylabel(r'$\epsilon$ / %')
    ax[0].invert_yaxis()

    ax[1].plot(sigma,eps(sigma,1000,10,0.9)*100)
    ax[1].set_xlabel(r"$\sigma'$ / kPa")
    ax[1].set_ylabel(r'$\epsilon$ / %')
    ax[1].invert_yaxis()
    ax[1].set_xscale('log')

    ax[2].plot(sigma,E_s(sigma,1000,10,0.9)/1e3)
    ax[2].set_xlabel(r"$\sigma'$ / kPa")
    ax[2].set_ylabel(r'$E_\mathrm{s}$ / MPa')
    fig.tight_layout();

In [3]:
def interactive_secant():
    @interact(
        s0 = widgets.BoundedFloatText(value=100,min=10,max=1000,description=r'$\sigma_0$ / kPa'),
        ds = widgets.BoundedFloatText(value=300,min=1,max=1000,description=r'$\Delta \sigma$ / kPa')        
    )
    def plot_secant(s0, ds):
        eps = lambda sig, Eref, sigref, om: sigref/((1-om)*Eref) * ( np.power(sig/sigref,1-om) -1 )
        sigma = np.linspace(10,1000,1000) #kPa
        eps0 = eps(s0, 1000, 10, 0.9)
        eps1 = eps(s0+ds, 1000, 10, 0.9)
        E_sec = ds/(eps1 - eps0)
        fig, ax = plt.subplots()
        ax.plot(sigma,eps(sigma,1000,10,0.9)*100)
        ax.plot([s0,s0+ds],[eps0*100,eps1*100],marker='o',color='red')
        ax.text(400,1,r'$E_\mathrm{s}^\mathrm{sec} = %.2f$ MPa' %(E_sec/1e3))
        ax.set_xlabel(r"$\sigma'$ / kPa")
        ax.set_ylabel(r'$\epsilon$ / %')
        ax.invert_yaxis()
        fig.tight_layout()
        plt.show();

In [ ]:
def plot_aushub():
    E_s = lambda sig, Eref, sigref, om: Eref * np.power(sig/sigref,om) #kPa
    eps = lambda sig, Eref, sigref, om: sigref*np.log(sig/sigref)/Eref if om == 1 else sigref/((1-om)*Eref) * ( np.power(sig/sigref,1-om) -1 )
    loopifier = lambda w: w*(1-w) * 0.005
    gamma = 16.
    q0 = 300.
    ts = 25.
    a = 6.
    b = 4.
    fig, ax = plt.subplots(figsize=(22,6),ncols=5)
    #original re-consolidation
    range0 = np.logspace(0,0.5,100)
    #virgin compression
    range1 = np.logspace(0.5,1.8,100)
    eps0_end = eps(range0[-1],1e3,1,0.9)
    eps1_beg = eps(range1[0],1e2,1,0.9)
    #unloading and reloading
    range2 = np.logspace(0.7,1.8,100)
    weight = (np.log(range2) - np.log(range2[0]))/(np.log(range2[-1]) - np.log(range2[0]))
    eps1_end = eps(range2[-1],1e2,1,0.9)+eps0_end-eps1_beg
    eps2_beg = eps(range2[-1],5e2,1,0.9)
    #virgin compression
    range3 = np.logspace(1.8,3,100)

    for i in range(5):
        ax[i].plot(range0,eps(range0,1e3,1,0.9),color='black')
        ax[i].plot(range1,eps(range1,1e2,1,0.9)+eps0_end-eps1_beg,color='black')

    for i in range(1,5):
        ax[i].plot(range2,eps(range2,5e2,1,0.9)+eps1_end-eps2_beg+loopifier(weight),color='red')

    for i in range(2,5):
        ax[i].plot(range2,eps(range2,5e2,1,0.9)+eps1_end-eps2_beg-loopifier(weight),color='green')

    for i in range(3,5):
        ax[i].plot(range3,eps(range3,1e2,1,0.9)+eps0_end-eps1_beg,color='blue')

    ax[0].set_title(r'Vor Aushub')
    ax[0].plot(range1[-1],eps1_end,marker='o',markersize='12',color='orange')

    ax[1].set_title(r'Nach Aushub')
    ax[1].plot(range2[0],eps(range2[0],5e2,1,0.9)+eps1_end-eps2_beg,marker='o',markersize='12',color='orange')

    ax[2].set_title(r'Fiktive Auffüllung')
    ax[2].plot(range2[-1],eps(range2[-1],5e2,1,0.9)+eps1_end-eps2_beg,marker='o',markersize='12',color='orange')

    ax[3].set_title(r'Nach Lastaufbringung')
    ax[3].plot(range3[-1],eps(range3[-1],1e2,1,0.9)+eps0_end-eps1_beg,marker='o',markersize='12',color='orange')

    ax[4].set_title(r'Sekantensteifemodul')
    ax[4].plot(range2[-1],eps(range2[-1],5e2,1,0.9)+eps1_end-eps2_beg,marker='o',markersize='12',color='orange')
    ax[4].plot(range3[-1],eps(range3[-1],1e2,1,0.9)+eps0_end-eps1_beg,marker='o',markersize='12',color='orange')
    ax[4].plot([range2[-1],range3[-1]],[eps(range2[-1],5e2,1,0.9)+eps1_end-eps2_beg,eps(range3[-1],1e2,1,0.9)+eps0_end-eps1_beg],ls='--',color='orange')


    for i in range(5):
        ax[i].invert_yaxis()
        if i < 4:
            ax[i].set_xscale('log')
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])
        ax[i].set_xlabel(r'$\sigma$')
        ax[i].set_ylabel(r'$\Delta h$')
    fig.suptitle(r'Einfluss der inkrementellen Nichtlinearität des Kompressionsverhaltens von Böden',fontweight='bold')
    fig.tight_layout();